In [40]:
%load_ext autoreload
%autoreload 2

import sys

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

sys.path.append("../modules")
from load_switrs import get_switrs_df, set_factorize
from highways import get_roads, get_road_codes, get_last_exit

roads = get_roads()
road_codes = get_road_codes()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df = get_switrs_df()
df.head(10)

,Case_ID,Jurisdiction,Officer_ID,Reporting_District,CHP_Shift,Population,County_City_Location,Special_Condition,Beat_Type,CHP_Beat_Type,...,Process_Date,Collision_Year,Collision_Month,Collision_Day,Collision_DayOfWeek,Collision_Hour,Collision_Minute,Collision_Hours,Collision_Minutes,Postmile_Code
0,2715171,9252,14755,None,2,9,3400,0,1,2,...,2006-11-01,2006,6,28,2,17,2,17.033333,1022.0,L0.75
1,2715231,9252,13560,None,2,9,3400,0,1,2,...,2006-11-13,2006,7,1,5,14,55,14.916667,895.0,R7.62
2,2715262,9252,14699,None,1,7,3404,0,1,2,...,2006-10-27,2006,6,26,0,9,55,9.916667,595.0,L1.01
3,2726805,9252,14977,None,2,7,3404,0,1,2,...,2006-11-01,2006,6,28,2,17,50,17.833333,1070.0,L0.7
4,2726822,9252,14512,None,2,7,3404,0,1,2,...,2006-11-01,2006,6,29,3,19,15,19.250000,1155.0,R4.65
5,2726830,9252,14512,None,2,9,3400,0,1,3,...,2006-11-01,2006,6,29,3,17,0,17.000000,1020.0,R16.01
6,2726833,9252,11681,None,1,9,3400,0,1,2,...,2006-11-01,2006,6,28,2,13,15,13.250000,795.0,R8.81
7,2726861,9252,14512,None,2,7,3404,0,1,2,...,2006-11-01,2006,6,30,4,16,20,16.333333,980.0,R1.19
8,2728728,9252,11342,None,1,9,3400,0,1,2,...,2006-11-01,2006,6,14,2,7,35,7.583333,455.0,R5.518
9,2728755,9252,14787,None,2,9,3400,0,1,2,...,2006-11-01,2006,6,26,0,17,30,17.500000,1050.0,L0.79


In [14]:
cols = ['State_Route', 'Caltrans_County', 'Postmile_Code']
df['Last_Exit'] = df[cols].apply(get_last_exit, axis=1)

df[cols + ['Last_Exit']].sample(20)

,State_Route,Caltrans_County,Postmile_Code,Last_Exit
210363,5,LA,R49.02,162
154449,15,SD,M16.36,0
119590,91,ORA,R14.24,36
22454,5,ORA,R24.02,101A
27141,12,SON,T18.24,0
93393,57,LA,R3.57,15
146947,134,LA,R7.93,08
91141,605,LA,R7.86,0
111116,215,RIV,R36.82,27B
41257,605,LA,R17.3,0


In [98]:
def get_highway_segment_df(row):
    return df[(df.State_Route     == row[0]) \
            & (df.Last_Exit       == row[1]) \
            & (df.Caltrans_County == row[2])]

In [99]:
def get_total_accidents(row):
    df = get_highway_segment_df(row)
    
    return len(df)

In [103]:
def get_avg_latitude(row):
    df = get_highway_segment_df(row)
        
    return df.Latitude.mean()

In [104]:
def get_avg_longitude(row):
    df = get_highway_segment_df(row)
        
    return df.Longitude.mean()

In [105]:
# Setup the highway segment data (split by highway)
df_highways = {}

for r in sorted(road_codes):
    # Start by loading Exit / County pairs
    df_r = df[(df.State_Route == r)][['Last_Exit', 'Caltrans_County']]

    exit_list = []
    # Useful function that filters to only unique pairs
    for e in np.vstack({tuple(row) for row in df_r.values}):
        if e[0] != '0':
            exit_list.append([r, e[0], e[1]])
            
    # If no pairs are found, something is horribly, horribly wrong!
    if len(exit_list) == 0:
        print 'Error Loading %6s!' % road_codes[r]
        continue
            
    # Otherwise, make the results into a DataFrame for each highway
    df_m = pd.DataFrame.from_records(
        exit_list,
        columns = ['Highway', 'Exit', 'County']
    )    
    
    # Then start calculating features for each one
    print 'Loading %6s...' % road_codes[r]
        
    df_m['Total_Accidents'] = df_m.apply(get_total_accidents, axis = 1)
    df_m['Avg_Latitude'] = df_m.apply(get_avg_latitude, axis = 1)
    df_m['Avg_Longitude'] = df_m.apply(get_avg_longitude, axis = 1)

    # And save the result for combining into the full set
    df_highways[r] = df_m.set_index('Exit')

Loading   CA-1...
Loading   CA-4...
Loading    I-5...
Error Loading   I-10!
Loading  CA-14...
Error Loading   I-15!
Loading  CA-22...
Error Loading  CA-24!
Error Loading   I-40!
Loading  CA-41...
Loading  US-50...
Error Loading  CA-55!
Loading  CA-57...
Loading  CA-58...
Loading  CA-60...
Error Loading   I-80!
Loading  CA-85...
Loading  CA-91...
Loading  CA-92...
Loading  CA-99...
Loading US-101...
Loading  I-105...
Loading CA-110...
Loading CA-118...
Loading CA-120...
Loading CA-126...
Loading CA-134...
Loading CA-170...
Loading CA-198...
Loading  I-205...
Error Loading  I-210!
Loading CA-215...
Loading  I-280...
Error Loading  I-605!
Loading  I-680...


In [ ]:
df_main_cols = [
    'Segment_ID', 'Highway', 'Last_Exit', 'Total_Accidents', 
    'County', 'County_Population', 
    'Morning_Commute_Accidents', 'Daytime_Accidents', 
    'Evening_Commute_Accidents', 'Nighttime_Accidents',
    'Interstate', 'Avg_Latitude', 'Avg_Longitude'
]

In [106]:
# Merge everything into one DataFrame for training / predicting
df_total = df_highways[1].copy()

for r in sorted(road_codes.keys()[1:]):
    if r in df_highways:
        df_total = df_total.append(df_highways[r])
        
county_list = set_factorize(df_total, 'County')
cl = dict(zip(county_list.values, range(len(county_list))))

In [107]:
for r in sorted(road_codes.keys()[1:]):
    if r in df_highways:
        d = df_highways[r]

        if 'County_old' not in d.columns:
            d['County_old'] = d.County
            d.County = d.County_old.apply(lambda x: cl[x])
    
df_highways[50].sample(20)

,Highway,County,Total_Accidents,Avg_Latitude,Avg_Longitude,County_old
Exit,,,,,,
05,50,19,986,38.539467,-121.484206,SAC
06A,50,19,233,38.547699,-121.482839,SAC
37,50,20,73,38.658345,-120.903647,ED
30,50,20,88,38.618689,-121.034942,ED
09,50,19,847,38.532835,-121.404140,SAC
54,50,20,38,38.744026,-120.643460,ED
57,50,20,31,38.693251,-120.586933,ED
41,50,20,54,38.693004,-120.880994,ED
49B,50,20,90,38.703944,-120.806924,ED


In [108]:
use_cols = ['Highway', 'County', 'Avg_Latitude', 'Avg_Longitude']

X = df_total[use_cols]
y = df_total.Total_Accidents

rf = RandomForestRegressor(
    n_estimators = 10000,
    verbose = 1,
)
rf.fit(X, y)
       
print "Features sorted by score:"
for s in sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), use_cols), reverse=True):
    print '\t%.4f - %s' % (s[0], s[1])

[Parallel(n_jobs=1)]: Done 10000 out of 10000 | elapsed:   13.3s finished


Features sorted by score:
	0.5423 - Avg_Latitude
	0.3392 - Avg_Longitude
	0.0690 - Highway
	0.0495 - County


In [112]:
df_test = df_highways[85].copy()

df_test['Predicted_Accidents'] = rf.predict(df_test[use_cols])

print df_test[['Total_Accidents', 'Predicted_Accidents']].sort_index()

      Total_Accidents  Predicted_Accidents
Exit                                      
09                 12              61.5907
10                 38             105.6801
11A                87             113.4426
11B               349             283.5940
14                227             208.4939
16                257             233.2544
18                186             167.7104
19A               109             141.4691
19B               203             177.9627
20                133             119.7914
22                 31              28.7951
22C                12              25.5682
23                  9              16.6685
24A                 5              12.3481


[Parallel(n_jobs=1)]: Done 10000 out of 10000 | elapsed:    4.3s finished
